# Create a MCQ Questions From a Given Context

In [3]:
!pip install pinecone-client

   ---------------------------------------- 0.0/214.5 kB ? eta -:--:--
   ----------- ---------------------------- 61.4/214.5 kB 1.7 MB/s eta 0:00:01
   ----------- ---------------------------- 61.4/214.5 kB 1.7 MB/s eta 0:00:01
   ---------------------------- ----------- 153.6/214.5 kB 1.3 MB/s eta 0:00:01
   ---------------------------- ----------- 153.6/214.5 kB 1.3 MB/s eta 0:00:01
   ---------------------------- ----------- 153.6/214.5 kB 1.3 MB/s eta 0:00:01
   ---------------------------- ----------- 153.6/214.5 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- 214.5/214.5 kB 654.6 kB/s eta 0:00:00


In [5]:
import openai
import pinecone
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import pinecone
from langchain.chains import RetrievalQA

In [6]:
def documentLoader(directory):
    loader= PyPDFDirectoryLoader(directory)
    docs = loader.load()
    return docs

In [8]:
docs = documentLoader('Docs/')
len(docs)

3

In [10]:
docs

[Document(page_content="India, officially known as the Republic of India, is a diverse and vibrant country located in South\nAsia. With a rich history spanning thousands of years, India is known for its cultural heritage, \nreligious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene\nbackwaters of Kerala in the south, India encompasses a wide range of geographical features, \nincluding deserts, plains, mountains, and coastlines, making it a land of incredible natural \nbeauty.\nIndia is the seventh-largest country by land area and the second-most populous country in the \nworld, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic \nrepublic, with a president as the head of state and a prime minister as the head of government. \nThe country follows a multi-tiered administrative structure, with 28 states and 9 union territories,\neach having its own elected government.\nIndia has a rich cultural heritage that has e

In [11]:
import os
os.environ['OPENAI_API_KEY'] = 'ENTER YOUR KEY' 
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'ENTER YOUR KEY' 

In [15]:
def split_docs(docs, chunk_size = 1000, chunk_overlap = 200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
    documents = text_splitter.split_documents(docs)
    return documents

In [16]:
documents = split_docs(docs)
len(documents)

9

In [18]:
documents[0]

Document(page_content='India, officially known as the Republic of India, is a diverse and vibrant country located in South\nAsia. With a rich history spanning thousands of years, India is known for its cultural heritage, \nreligious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene\nbackwaters of Kerala in the south, India encompasses a wide range of geographical features, \nincluding deserts, plains, mountains, and coastlines, making it a land of incredible natural \nbeauty.\nIndia is the seventh-largest country by land area and the second-most populous country in the \nworld, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic \nrepublic, with a president as the head of state and a prime minister as the head of government. \nThe country follows a multi-tiered administrative structure, with 28 states and 9 union territories,\neach having its own elected government.', metadata={'source': 'Docs\\Doc 1.pdf', 'page'

In [19]:
embeddings = SentenceTransformerEmbeddings()

In [25]:
query_result = embeddings.embed_query('India, officially known as the Republic of India, is a diverse and vibrant country located in South\nAsia. With a rich history spanning thousands of years, India is known for its cultural heritage, \nreligious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene\nbackwaters of Kerala in the south, India encompasses a wide range of geographical features, \nincluding deserts, plains, mountains, and coastlines, making it a land of incredible natural \nbeauty.\nIndia is the seventh-largest country by land area and the second-most populous country in the \nworld, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic \nrepublic, with a president as the head of state and a prime minister as the head of government. \nThe country follows a multi-tiered administrative structure, with 28 states and 9 union territories,\neach having its own elected government')

In [26]:
len(query_result)

768

In [27]:
from pinecone import Pinecone as PineconeClient
from langchain_community.vectorstores import Pinecone

In [30]:
os.environ["PINECONE_API_KEY"] = <>
PINECONE_API_KEY=os.getenv("‘PINECONE_API_KEY’")

PineconeClient(api_key=PINECONE_API_KEY, environment="gcp-starter")
index_name="mcq"
index = Pinecone.from_documents(documents, embeddings, index_name=index_name)

In [31]:
def similar_docs(query, k=2):
    similar_docs = index.similarity_search(query, k=k)
    return similar_docs

In [32]:
from langchain.chains.question_answering import load_qa_chain

In [33]:
from langchain.llms import HuggingFaceEndpoint

In [34]:
llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2") # Model link : https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2
llm

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\pushp\.cache\huggingface\token
Login successful


HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>)

In [35]:
chain = load_qa_chain(llm=llm, chain_type='stuff')

In [62]:
def get_answer(query):
    similar_documents = similar_docs(query)
    print(similar_documents)
    response = chain.run(input_documents = similar_documents, question = query)
    print(response)
    return response
    

In [63]:
answer = get_answer("how is india's economy ?")

[Document(page_content="India's diplomatic influence is also growing on the global stage. The country actively \nparticipates in international forums and has strong bilateral relations with nations around the \nworld. India is a founding member of the Non-Aligned Movement and plays an active role in \nvarious international organizations, such as the United Nations and World Trade Organization.\nIn conclusion, India is a vast and diverse country with a rich cultural heritage, stunning \nlandscapes, and a rapidly growing economy. It is a nation where ancient traditions coexist with \nmodern aspirations. Despite its challenges, India continues to evolve and leave an indelible \nmark on the world, making it a fascinating and dynamic country to explore.", metadata={'page': 1.0, 'source': 'Docs\\Doc 2.pdf'}), Document(page_content="India's diplomatic influence is also growing on the global stage. The country actively \nparticipates in international forums and has strong bilateral relations w

In [64]:
answer

" India's economy is rapidly growing. India is a member of the World Trade Organization and plays an active role in international organizations such as the United Nations. The country has strong bilateral relations with nations around the world and is a founding member of the Non-Aligned Movement. Despite challenges, India continues to evolve and leave an indelible mark on the world."

In [47]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [48]:
response_schemas = [
    ResponseSchema(name="question", description="Question generated from provided input text data."),
    ResponseSchema(name="choices", description="Available options for a multiple-choice question in comma separated."),
    ResponseSchema(name="answer", description="Correct answer for the asked question.")
]

In [50]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas=response_schemas)

In [52]:
print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"question": string  // Question generated from provided input text data.
	"choices": string  // Available options for a multiple-choice question in comma separated.
	"answer": string  // Correct answer for the asked question.
}
```


In [ ]:
]

In [41]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [42]:
prompt = PromptTemplate(template="How are you {name} ?",
                       input_variables=['name'])

In [44]:
print(prompt.format_prompt(name='Pushpjeet'))

text='How are you Pushpjeet ?'


In [56]:
prompt = ChatPromptTemplate(messages = [HumanMessagePromptTemplate.from_template
    ("""When a text input is given by the user,please generate multiple choice questions from it along with 
    the correct answer. \n{format_instructions}\n{user_prompt}""")],
                           input_variables=['user_prompt'],
                           partial_variables={'format_instructions':output_parser.get_format_instructions()})

In [83]:
ans = """User Story:
As a lead manager,
I need to efficiently filter leads by business category and user,
So that I can focus on relevant data for targeted analysis and action.

Acceptance Criteria:

Given that I'm logged in, the lead screen should provide two filters: Business Category and User.
The User filter should dynamically display a list of users reporting directly or indirectly to me.
Both filters should offer an "All" option and support single selection only.
Upon selecting a Business Category, the User filter should automatically default to "All".
Clicking on the User filter should present only the users associated with the selected segment.
When choosing a user, the category filter should adjust to display categories related to the selected user.
Clicking on the category filter should present "All" and other category relevant to the logged-in user, regardless of the user selection from the dropdown.
Additional Details:

Ensure smooth filter interaction and real-time updates to streamline the user experience.
Provide clear visual cues for selected filters and their respective options.
Maintain consistency in filter behavior across different user interactions."""

In [84]:
final_query = prompt.format_prompt(user_prompt=ans)

In [85]:
print(final_query)

messages=[HumanMessage(content='When a text input is given by the user,please generate multiple choice questions from it along with \n    the correct answer. \nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"question": string  // Question generated from provided input text data.\n\t"choices": string  // Available options for a multiple-choice question in comma separated.\n\t"answer": string  // Correct answer for the asked question.\n}\n```\nUser Story:\nAs a lead manager,\nI need to efficiently filter leads by segment and user,\nSo that I can focus on relevant data for targeted analysis and action.\n\nAcceptance Criteria:\n\nGiven that I\'m logged in, the lead screen should provide two filters: Segment and User.\nThe User filter should dynamically display a list of users reporting directly or indirectly to me.\nBoth filters should offer an "All" option and support single selection

In [86]:
print(final_query.to_messages())

[HumanMessage(content='When a text input is given by the user,please generate multiple choice questions from it along with \n    the correct answer. \nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"question": string  // Question generated from provided input text data.\n\t"choices": string  // Available options for a multiple-choice question in comma separated.\n\t"answer": string  // Correct answer for the asked question.\n}\n```\nUser Story:\nAs a lead manager,\nI need to efficiently filter leads by segment and user,\nSo that I can focus on relevant data for targeted analysis and action.\n\nAcceptance Criteria:\n\nGiven that I\'m logged in, the lead screen should provide two filters: Segment and User.\nThe User filter should dynamically display a list of users reporting directly or indirectly to me.\nBoth filters should offer an "All" option and support single selection only.\nU

In [87]:
from langchain_openai import ChatOpenAI

In [88]:
chat_model = ChatOpenAI()

In [89]:
final_query_output = chat_model.invoke(final_query.to_messages())

In [90]:
print(final_query_output.content)

```json
{
	"question": "What should the lead screen provide for filtering leads?",
	"choices": "Segment and User, User and Project, Segment and Project, User and Team",
	"answer": "Segment and User"
}
```

```json
{
	"question": "What should the User filter dynamically display?",
	"choices": "All users, Only direct reports, Only indirect reports, Only managers",
	"answer": "All users"
}
```

```json
{
	"question": "What should happen when a segment is selected?",
	"choices": "User filter defaults to 'All', User filter displays only selected segment users, User filter resets to blank, No change in User filter",
	"answer": "User filter defaults to 'All'"
}
```

```json
{
	"question": "What should clicking on the User filter present?",
	"choices": "All users, Only managers, Only team leads, Only sales reps",
	"answer": "Only the users associated with the selected segment"
}
```

```json
{
	"question": "How should the Segment filter adjust when choosing a user?",
	"choices": "Display all s

In [91]:
import re

In [92]:
markdown_text = final_query_output.content

In [96]:
print(re.search(r'{(.*?)}',  markdown_text, re.DOTALL).group(1))


	"question": "What should the lead screen provide for filtering leads?",
	"choices": "Segment and User, User and Project, Segment and Project, User and Team",
	"answer": "Segment and User"

